# Sun Cloud Pavement

Join together various pavement condition layers into a single layer with standardized good / fair / poor ratings.

Refer to the data layer documentation for crosswalk methodology and sources.

The cells below are used to calculated a standardized "sun_cloud_condition" good/fair/poor rating for each pavement section. Then, the layers are combined.

Sources:

* ADOT
* MAG (ABNA)
* Tucson
* Pima County
* SC MPO (manual shapefile creation from spreadsheets)
* SEAGO

In [ ]:
# Add a copy of Sun Cloud Routes from AZ Geo and then
# export to the local working GDB
arcpy.conversion.ExportFeatures(
    "Sun Cloud Routes", 
    r"pavement_layer.gdb\sun_cloud_routes", 
    
)

In [1]:
arcpy.management.CalculateField(
    "ADOT_pavement_2021", 
    "source", 
    "'Arizona DOT (2021)'", 
    "PYTHON3", '', "TEXT"
)

<Result 'ADOT_pavement_2021'>

In [2]:
arcpy.management.CalculateField(
    "ABNA_pavement_condition", 
    "source", 
    "'MAG ABNA (2019)'", 
    "PYTHON3", '', "TEXT"
)

<Result 'ABNA_pavement_condition'>

In [3]:
arcpy.management.CalculateField(
    "Tucson_pavement_condition", 
    "source", 
    "'City of Tucson'", 
    "PYTHON3", '', "TEXT"
)

<Result 'Tucson_pavement_condition'>

In [4]:
arcpy.management.CalculateField(
    "PIMA_pavement_condition", 
    "source", 
    "'Pima County'", 
    "PYTHON3", '', "TEXT"
)

<Result 'PIMA_pavement_condition'>

In [5]:
arcpy.management.CalculateField(
    "scmpo_manual_roads", 
    "source", 
    "'Sun Corridor MPO (2019)'", 
    "PYTHON3", '', "TEXT"
)

<Result 'scmpo_manual_roads'>

In [6]:
arcpy.management.CalculateField(
    "SEAGO_pavement_condition", 
    "source", 
    "'SEAGO (2022)'", 
    "PYTHON3", '', "TEXT"
)

<Result 'SEAGO_pavement_condition'>

In [ ]:
arcpy.management.CalculateField(
    "ABNA_pavement_condition", 
    "sun_cloud_condition", 
    "eval_pci(!pci!)", 
    "PYTHON3", 
"""def eval_pci(pci):
    if pci is None:
        return None
    elif pci <= 55:
        return 'Poor'
    elif pci <= 70:
        return 'Fair'
    elif pci <= 100:
        return 'Good'
    else:
        return 'Unknown'
""", "TEXT")

In [2]:
# Pima County
arcpy.management.CalculateField(
    "PIMA_pavement_condition", 
    "sun_cloud_condition", 
    "eval_pci(!DPASER!)", 
    "PYTHON3", 
"""def eval_pci(pci):
    if pci is None:
        return None
    elif pci <= 55:
        return 'Poor'
    elif pci <= 70:
        return 'Fair'
    elif pci <= 100:
        return 'Good'
    else:
        return 'Unknown'
""", "TEXT")

<Result 'PIMA_pavement_condition'>

In [1]:
# Tucson
arcpy.management.CalculateField(
    "Tucson_pavement_condition", 
    "sun_cloud_condition", 
    "eval_oci(!OCI!)", 
    "PYTHON3", 
"""def eval_oci(oci):
    if oci is None:
        return None
    elif oci < 55:
        return 'Poor'
    elif oci < 70:
        return 'Fair'
    elif oci <= 100:
        return 'Good'
    else:
        return 'Unknown'
""", "TEXT")

<Result 'Tucson_pavement_condition'>

In [3]:
# SEAGO
# based on cutoffs from John Meredith
arcpy.management.CalculateField(
    "SEAGO_pavement_condition", 
    "sun_cloud_condition", 
    "eval(!PavemCondi!)", 
    "PYTHON3", 
"""def eval(pct):
    if pct is None:
        return None
    elif pct < .50:
        return 'Poor'
    elif pct < .60:
        return 'Fair'
    elif pct <= 1.00:
        return 'Good'
    else:
        return 'Unknown'
""", "TEXT")

<Result 'SEAGO_pavement_condition'>

In [1]:
# SCMPO
arcpy.management.CalculateField(
    "scmpo_manual_roads", 
    "sun_cloud_condition", 
    "eval_paser(!condition_2019!)", 
    "PYTHON3", 
"""def eval_paser(cond):
    if cond is None:
        return None
    elif cond <= 3:
        return 'Poor'
    elif cond <= 5:
        return 'Fair'
    elif cond <= 10:
        return 'Good'
    else:
        return 'Unknown'
""", "TEXT")

<Result 'scmpo_manual_roads'>

**note, ADOT's data layer includes a good / fair / poor rating based on the FHWA definition. This is used as is.**

## Combine and Conflate

1. Merge SCMPO, SEAGO, Tucson, PIMA --> Merged
2. Unsplit ADOT, ABNA, and Merged
3. Join Merged to Sun Cloud Routes
4. Combine

In [8]:
# merge today layers for conflation to ATIS linework
arcpy.management.Merge(
    "ABNA_pavement_condition;Tucson_pavement_condition;SEAGO_pavement_condition;scmpo_manual_roads;PIMA_pavement_condition", 
    r"pavement_layer.gdb\non_adot_merge", 
    'sun_cloud_condition "sun_cloud_condition" true true false 512 Text 0 0,First,#,ABNA_pavement_condition,sun_cloud_condition,0,512,Tucson_pavement_condition,sun_cloud_condition,0,512,SEAGO_pavement_condition,sun_cloud_condition,0,512,scmpo_manual_roads,sun_cloud_condition,0,512,PIMA_pavement_condition,sun_cloud_condition,0,512;source "source" true true false 512 Text 0 0,First,#,ABNA_pavement_condition,source,0,512,Tucson_pavement_condition,source,0,512,SEAGO_pavement_condition,source,0,512,scmpo_manual_roads,source,0,512,PIMA_pavement_condition,source,0,512', 
    "NO_SOURCE_INFO"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\pavement\\pavement_layer.gdb\\non_adot_merge'>

In [10]:
arcpy.management.UnsplitLine(
    "non_adot_merge", 
    "others_unsplit", 
    "sun_cloud_condition;source", 
    None, '')

<Result 'Y:\\mag\\sun_cloud\\layers\\pavement\\pavement_layer.gdb\\others_unsplit'>

In [11]:
# break ROUTES where pavement condition splits
# 1. create points at the end of pavement condition sections
# 2. snap points to ROUTES
# 3. break ROUTES at POINTS
arcpy.management.GeneratePointsAlongLines(
    "others_unsplit", 
    "UnsplitLine_GeneratePointsAlongLines", 
    "PERCENTAGE", None, 100, 
    "END_POINTS"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\pavement\\pavement_layer.gdb\\UnsplitLine_GeneratePointsAlongLines'>

In [13]:
arcpy.edit.Snap(
    "UnsplitLine_GeneratePointsAlongLines", 
    "sun_cloud_routes EDGE '50 Meters'"
)

<Result 'UnsplitLine_GeneratePointsAlongLines'>

In [14]:
arcpy.management.SplitLineAtPoint(
    "sun_cloud_routes", 
    "UnsplitLine_GeneratePointsAlongLines", 
    r"pavement_layer.gdb\routes_split", 
    "1 Meters"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\pavement\\pavement_layer.gdb\\routes_split'>

In [15]:
arcpy.analysis.PairwiseBuffer(
    "others_unsplit", 
    r"pavement_layer.gdb\others_unsplit_PairwiseBuffe", 
    "10 Meters", "NONE", None, 
    "PLANAR", "0 Feet"
)

<Result 'Y:\\mag\\sun_cloud\\layers\\pavement\\pavement_layer.gdb\\others_unsplit_PairwiseBuffe'>

In [17]:
arcpy.management.AddSpatialJoin(
    "routes_split", 
    "others_unsplit_PairwiseBuffe", 
    "JOIN_ONE_TO_ONE", 
    "KEEP_ALL"
)

<Result 'routes_split'>

In [ ]:
# extract spatial join result
arcpy.conversion.ExportFeatures("routes_split", r"Y:\mag\sun_cloud\layers\pavement\pavement_layer.gdb\routes_split_not_null", '', "NOT_USE_ALIAS", 'route_id "Route ID" true true false 32 Text 0 0,First,#,routes_split,routes_split.route_id,0,32;functional_class "Functional Classification" true true false 4 Long 0 0,First,#,routes_split,routes_split.functional_class,-1,-1;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,routes_split,routes_split.ORIG_FID,-1,-1;ORIG_SEQ "ORIG_SEQ" true true false 4 Long 0 0,First,#,routes_split,routes_split.ORIG_SEQ,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,routes_split,routes_split.Shape_Length,-1,-1;OBJECTID "OBJECTID" false true false 4 Long 0 9,First,#,routes_split,routes_split_AddSpatialJoin_2.OBJECTID,-1,-1;Join_Count "Join_Count" true true false 4 Long 0 0,First,#,routes_split,routes_split_AddSpatialJoin_2.Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,routes_split,routes_split_AddSpatialJoin_2.TARGET_FID,-1,-1;sun_cloud_condition "sun_cloud_condition" true true false 512 Text 0 0,First,#,routes_split,routes_split_AddSpatialJoin_2.sun_cloud_condition,0,512;source "source" true true false 512 Text 0 0,First,#,routes_split,routes_split_AddSpatialJoin_2.source,0,512;BUFF_DIST "BUFF_DIST" true true false 8 Double 0 0,First,#,routes_split,routes_split_AddSpatialJoin_2.BUFF_DIST,-1,-1;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,routes_split,routes_split_AddSpatialJoin_2.ORIG_FID,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,routes_split,routes_split_AddSpatialJoin_2.Shape_Length,-1,-1', None)

In [ ]:
# subtract out ADOT roads from others
arcpy.analysis.PairwiseErase("routes_split_not_null", "ADOT_pavement_2021", r"Y:\mag\sun_cloud\layers\pavement\pavement_layer.gdb\routes_split_n_PairwiseErase", None)

In [ ]:
# merge ADOT and others
arcpy.management.Merge("routes_split_n_PairwiseErase;ADOT_pavement_2021", r"Y:\mag\sun_cloud\layers\pavement\pavement_layer.gdb\routes_split_n_Pairwis_Merge", 'route_id "Route ID" true true false 32 Text 0 0,First,#,routes_split_n_PairwiseErase,route_id,0,32,ADOT_pavement_2021,route_id,0,32;sun_cloud_condition "sun_cloud_condition" true true false 512 Text 0 0,First,#,routes_split_n_PairwiseErase,sun_cloud_condition,0,512,ADOT_pavement_2021,sun_cloud_condition,0,512;source "source" true true false 512 Text 0 0,First,#,routes_split_n_PairwiseErase,source,0,512,ADOT_pavement_2021,source,0,512', "NO_SOURCE_INFO")

In [ ]:
# select not null and unsplit
arcpy.management.SelectLayerByAttribute("routes_split_n_Pairwis_Merge", "NEW_SELECTION", "sun_cloud_condition IS NOT NULL", None)
arcpy.management.UnsplitLine("routes_split_n_Pairwis_Merge", r"Y:\mag\sun_cloud\layers\pavement\pavement_layer.gdb\routes_split_n_P_UnsplitLine", "route_id;sun_cloud_condition;source", None, '')

In [ ]:
# omit < 25' long and export
arcpy.management.SelectLayerByAttribute("routes_split_n_P_UnsplitLine", "NEW_SELECTION", "Shape_Length > 25", None)
arcpy.conversion.ExportFeatures("routes_split_n_P_UnsplitLine", r"Y:\mag\sun_cloud\layers\pavement\pavement_layer.gdb\Sun_Cloud_Combined_Pavement", '', "NOT_USE_ALIAS", 'route_id "Route ID" true true false 32 Text 0 0,First,#,routes_split_n_P_UnsplitLine,route_id,0,32;sun_cloud_condition "sun_cloud_condition" true true false 512 Text 0 0,First,#,routes_split_n_P_UnsplitLine,sun_cloud_condition,0,512;source "source" true true false 512 Text 0 0,First,#,routes_split_n_P_UnsplitLine,source,0,512;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,routes_split_n_P_UnsplitLine,Shape_Length,-1,-1', None)

In [19]:
# KEEP AND RENAME FIELDS
working_layer = 'Sun_Cloud_Combined_Pavement'

system_fields = [arcpy.Describe(working_layer).OIDFieldName, 'Shape', 'geom', 'Shape_Length']
current_fields = [f.name for f in arcpy.ListFields(working_layer) if not f.name in system_fields]

print(', '.join(current_fields))

# Define tuples of CurrentFieldName, target_field_name, Alias
target_fields = [
    ('route_id', 'route_id', 'Route ID'),
    ('sun_cloud_condition', 'sun_cloud_condition', 'Pavement Condition'),
    ('source', 'source', 'Pavement Condition Information Source')
]

# delete unused fields
delete_fields = [field for field in current_fields if not field in [a for a, b, c in target_fields]]
if delete_fields:
    arcpy.management.DeleteField(working_layer, delete_fields)
    
for current, new, alias in target_fields:
    # arcgis can't change capitalization ... a hacky workaround
    if new != current and new.lower() == current.lower():
        intermediate = current + "1"
        arcpy.management.AlterField(working_layer, current, intermediate)
        current = intermediate
    arcpy.management.AlterField(working_layer, current, new, alias)


route_id, sun_cloud_condition, source
